In [84]:
%matplotlib inline

In [1]:
import argparse
import tempfile
import time
from typing import List

import pandas
import pyarrow

import ray
from ray.data.dataset_pipeline import DatasetPipeline

import numpy as np
import tensorflow as tf

2022-02-17 15:51:57.006407: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-02-17 15:51:57.006435: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def mnist_dataset(batch_size):
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # The `x` arrays are in uint8 and have values in the [0, 255] range.
    # You need to convert them to float32 with values in the [0, 1] range.
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64) 
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
    return train_dataset

In [ ]:
# =======================================

In [2]:
def mnist_dataset_ray_distributed():
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    # The `x` arrays are in uint8 and have values in the [0, 255] range.
    # You need to convert them to float32 with values in the [0, 1] range.
    x_train = x_train / np.float32(255)
    y_train = y_train.astype(np.int64)
    
    x_2d = x_train.reshape(len(x_train), 28*28) 
    y_train_re = y_train.reshape(-1, 1)   
    df = pandas.DataFrame(x_2d)
    df['y'] = y_train_re

    return ray.data.from_pandas(df)

In [3]:
def create_shuffle_pipeline(num_epochs: int,num_wokers: int) -> List[DatasetPipeline]:
    datasource = mnist_dataset_ray_distributed()
    return datasource.repeat(num_epochs) \
        .random_shuffle_each_window() \
        .split(num_wokers, equal=True)

In [4]:
@ray.remote
class TrainingWorker:
    def __init__(self, rank: int, shard: DatasetPipeline):
        self.rank = rank
        self.shard = shard
        self.model = self.build_cnn_model()

    def train(self):
        for epoch, training_dataset in enumerate(self.shard.iter_epochs()):
            # Following code emulates epoch based SGD training.
            print(f"Training... worker: {self.rank}, epoch: {epoch}")
            for i, batch in enumerate(training_dataset.iter_batches(batch_format="pandas")):
                # TODO: replace the code for real training. 
                trans_x = batch[batch.columns.difference(['y'])].to_numpy()  
                x_3d = np.reshape(trans_x, (len(trans_x), 28, 28))
                y_1d = np.reshape(batch['y'].to_numpy(), len(batch['y']))
                self.model.fit(x_3d, y_1d, epochs=1)

    def build_cnn_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(28, 28)),
            tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(32, 3, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])
        model.compile(
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
            metrics=['accuracy'])
        return model 

In [5]:
# create a connection to a ray cluster.
ray.init("ray://10.100.3.60:10001")

ClientContext(dashboard_url='192.168.95.118:8265', python_version='3.7.7', ray_version='1.10.0', ray_commit='5ea565317a8104c04ae7892bb9bb41c6d72f12df', protocol_version='2021-12-07', _num_clients=1, _context_to_restore=<ray.util.client._ClientContext object at 0x7f6fd60dcf50>)

In [6]:
NUM_TRAINING_WORKERS = 3
NUM_EPOCHS = 2 

In [7]:
splits = create_shuffle_pipeline(NUM_EPOCHS,NUM_TRAINING_WORKERS)

Stage 1:   0%|          | 0/2 [00:00<?, ?it/s]90) 


In [8]:
training_workers = [
    TrainingWorker.remote(rank, shard) for rank, shard in enumerate(splits)
]

(TrainingWorker pid=847, ip=192.168.30.1) 2022-02-17 07:52:25.567465: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
(TrainingWorker pid=847, ip=192.168.30.1) 2022-02-17 07:52:25.567487: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
(TrainingWorker pid=847, ip=192.168.30.1) 2022-02-17 07:52:25.567510: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ray-test-cluster-ray-worker-type-z4scc): /proc/driver/nvidia/version does not exist
(TrainingWorker pid=847, ip=192.168.30.1) 2022-02-17 07:52:25.567697: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU ins

In [9]:
# Let's run the e2e pipeline
start = time.time()

print(ray.get([worker.train.remote() for worker in training_workers]))

print("total ingestion time: {%.4f}s" % float(time.time() - start))

Stage 0:  50%|█████     | 1/2 [00:13<00:13, 13.25s/it]0m 
(PipelineSplitExecutorCoordinator pid=3590) 
Stage 0: 100%|██████████| 2/2 [00:14<00:00,  6.45s/it]
(PipelineSplitExecutorCoordinator pid=3590) 
Stage 1: 100%|██████████| 2/2 [00:17<00:00,  7.44s/it]


(TrainingWorker pid=267, ip=192.168.43.62) Training... worker: 0, epoch: 0
(TrainingWorker pid=268, ip=192.168.43.62) Training... worker: 2, epoch: 0


Stage 0:   0%|          | 0/1 [00:00<?, ?it/s]0.1) 


(TrainingWorker pid=847, ip=192.168.30.1) Training... worker: 1, epoch: 0


(TrainingWorker pid=267, ip=192.168.43.62) 2022-02-17 07:52:38.126703: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


...

Stage 0: 100%|██████████| 1/1 [00:16<00:00, 16.11s/it] 


  7/625 [..............................] - ETA: 36s - loss: 1.4889 - accuracy: 0.7411
(TrainingWorker pid=267, ip=192.168.43.62)  - ETA: 26s - loss: 1.6487 - accuracy: 0.7296
 11/625 [..............................].168.43.62)
(TrainingWorker pid=268, ip=192.168.43.62)  - ETA: 34s - loss: 1.5082 - accuracy: 0.7301
 88/625 [===>..........................] - ETA: 27s - loss: 1.4740 - accuracy: 0.7525
(TrainingWorker pid=267, ip=192.168.43.62)  - ETA: 24s - loss: 1.5804 - accuracy: 0.7379
211/625 [=========>....................] - ETA: 20s - loss: 1.3678 - accuracy: 0.7657
(TrainingWorker pid=268, ip=192.168.43.62)
251/625 [===========>..................] - ETA: 18s - loss: 1.4726 - accuracy: 0.7552
(TrainingWorker pid=268, ip=192.168.43.62) 213/625 [=========>....................] - ETA: 20s - loss: 1.3660 - accuracy: 0.7660
587/625 [===========================>..] - ETA: 1s - loss: 1.1015 - accuracy: 0.8000


Stage 0: 100%|██████████| 1/1 [00:32<00:00, 32.20s/it]m 


619/625 [============================>.] - ETA: 0s - loss: 1.0854 - accuracy: 0.8018
[None, None, None]
total ingestion time: {70.7700}s
625/625 [==============================] - 31s 49ms/step - loss: 1.0825 - accuracy: 0.8021


Stage 0: 100%|██████████| 1/1 [00:31<00:00, 31.38s/it]m 


In [10]:
# Connect to a cluster
ray.shutdown()

In [ ]:
# Test Code =================================

In [ ]:
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
# The `x` arrays are in uint8 and have values in the [0, 255] range.
# You need to convert them to float32 with values in the [0, 1] range.
x_train = x_train / np.float32(255)
y_train = y_train.astype(np.int64)

print(len(x_train), x_train.shape) 
print(len(x_train[0]), x_train[0].shape)
print(len(x_train[0][0]), x_train[0][0].shape)

x_2d = x_train.reshape(len(x_train), 28*28)
print(len(x_2d), x_2d.shape)

y_train_re = y_train.reshape(-1, 1)  
print(len(y_train), y_train.shape)
print(len(y_train_re), y_train_re.shape)

df = pandas.DataFrame(x_2d)
df['y'] = y_train_re
y_1d = np.reshape(df['y'].to_numpy(), len(df['y']))
print(df['y'], df['y'].shape)
print(y_1d, y_1d.shape)

print(df, df.shape)

trans_x = df[df.columns.difference(['y'])]
trans_x_np = trans_x.to_numpy()
print(trans_x_np, trans_x_np.shape)

x_3d = np.reshape(trans_x_np, (len(trans_x), 28, 28))
print(x_3d, x_3d.shape)
print(x_3d[0], x_3d[0].shape)